In [1]:
import pandas as pd
import numpy as np
import os
os.listdir('../my/data')

['BC_1022.csv',
 'BC_32ea_k100_cyto.csv',
 'BC_32ea_k15_cyto.csv',
 'K15_1022.csv',
 'K15_common.csv',
 'normal_1022.csv',
 'Normal_21ea_k100_cyto.csv',
 'Normal_21ea_k15_cyto.csv',
 'PC_1022.csv',
 'PC_20ea_k100_cyto.csv',
 'PC_20ea_k15_cyto.csv',
 'RC_20ea_k100_cyto.csv',
 'TCGA_common.csv',
 'TCGA_data.csv']

In [2]:
#NA
tcga = pd.read_csv('../my/data/TCGA_common.csv')
urine = pd.read_csv('../my/data/K15_common.csv')

urine_BC_Norm=urine.loc[urine['cancer'].isin(['Normal','BC'])]
tcga_BC_Norm=tcga.loc[tcga['y'].isin(['Normal','BC'])]
X_train = tcga_BC_Norm.drop('y', axis = 1)
y_train = tcga_BC_Norm['y']
X_test = urine_BC_Norm.drop('cancer',axis=1)
y_test= urine_BC_Norm['cancer']

print('X_train shape : , ', X_train.shape, 'y_train : ', y_train.shape)
print('X_test shape : , ', X_test.shape, 'y_test : ', y_test.shape)

X_train shape : ,  (2114, 763) y_train :  (2114,)
X_test shape : ,  (53, 763) y_test :  (53,)


In [3]:
y_train[y_train == 'BC'] =1
y_train[y_train == 'Normal'] = 0

y_test[y_test == 'BC'] = 1
y_test[y_test == 'Normal'] = 0

y_train = y_train.astype('int')
y_test = y_test.astype('int')

C:\Users\wjssm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\wjssm\Anaconda3\lib\site-packages\pandas\core\generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\wjssm\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C

In [4]:
y_train.value_counts()

0    1706
1     408
Name: y, dtype: int64

In [5]:
y_test.value_counts()

1    32
0    21
Name: cancer, dtype: int64

In [6]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve

def get_eval(y_test, pred) :
    
    print('accuracy : {0:.4f} precision : {1:.4f} recall : {2:.4f} F1 : {3:.4f} AUC : {4:.4f}'.format(
                                                        accuracy_score(y_test, pred), 
                                                        precision_score(y_test, pred),
                                                        recall_score(y_test, pred),
                                                        f1_score(y_test,pred),
                                                        roc_auc_score(y_test, pred)
                                                                      ))
    
    
    
    #print('accuracy : {0:.4f}'.format(accuracy_score(y_test,pred)), 'roc_auc_score : ', roc_auc_score(y_test,pred))
    #print(classification_report(y_test, pred))
    print(confusion_matrix(y_test, pred))

In [7]:
import matplotlib.pyplot as plt

#draw roc curve
def roc_curve_plot(y_test, pred_proba) :
    
    fpr, tpr, threshold = roc_curve(y_test, pred_proba)
    plt.plot(fpr, tpr, label = 'ROC')
    plt.plot([0,1],[0,1],'k--', label = 'Random')
    
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1)
    plt.ylim(0,1)
    
    plt.xlabel('FPR(1-Sensitivity)')
    plt.ylabel('TPR(Recall)')
    plt.legend()

# scaling

In [19]:
X_test = X_test[X_train.columns]
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler
#MinMaxScaler
X_train_mms = MinMaxScaler().fit_transform(X_train)
X_test_mms = MinMaxScaler().fit_transform(X_test)

#MaxAbsScaler
X_train_mas = MaxAbsScaler().fit_transform(X_train)
X_test_mas = MaxAbsScaler().fit_transform(X_test)

In [20]:
X_train_mas.columns

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [9]:
import warnings
warnings.filterwarnings('ignore')

# voting

In [10]:
from sklearn.ensemble import VotingClassifier

def voting_func(models, X_train, X_test) :

    soft_vote = VotingClassifier(estimators = models, voting = 'soft')
    hard_vote = VotingClassifier(estimators = models, voting = 'hard')

    print('soft voting')
    soft_vote.fit(X_train ,y_train)
    vote_pred = soft_vote.predict(X_test)
    get_eval(y_test, vote_pred)

    print('hard voting')
    hard_vote.fit(X_train, y_train)
    vote_pred = hard_vote.predict(X_test)
    get_eval(y_test, vote_pred)

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

## Original data

In [11]:
#DecisionTree
dt = DecisionTreeClassifier(random_state = 156, max_depth = 8)
dt.fit(X_train, y_train)

#RandomForest
rf = RandomForestClassifier(n_jobs = -1, max_depth=7, min_samples_leaf=8, min_samples_split=16,
                                n_estimators = 200)
rf.fit(X_train, y_train)

#SVM
svc = SVC(kernel='rbf', probability=True)
svc.fit(X_train, y_train)

#LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

#GBM
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [12]:
voting_func([('DT', dt), ('RF', rf), ('SVM',svc), ('LR', lr), ('GB', gb)], X_train, X_test)

soft voting
accuracy : 0.6038 precision : 0.6222 recall : 0.8750 F1 : 0.7273 AUC : 0.5327
[[ 4 17]
 [ 4 28]]
hard voting
accuracy : 0.5849 precision : 0.6042 recall : 0.9062 F1 : 0.7250 AUC : 0.5007
[[ 2 19]
 [ 3 29]]


## MinMaxScale

In [26]:
#DecisionTree
dt = DecisionTreeClassifier(random_state = 156, max_depth = 8)
dt.fit(X_train_mms, y_train)

#RandomForest
rf = RandomForestClassifier(n_jobs = -1, max_depth=7, min_samples_leaf=8, min_samples_split=16,
                                n_estimators = 200)
rf.fit(X_train_mms, y_train)

#SVM
svc = SVC(kernel='rbf', probability=True)
svc.fit(X_train_mms, y_train)


lr = LogisticRegression()
lr.fit(X_train_mms, y_train)

gb = GradientBoostingClassifier()
gb.fit(X_train_mms, y_train)

xgb = XGBClassifier(learning_rate=0.001, n_estimators=400, n_jobs=-1, max_depth = 7)
evals = [(X_test_mms, y_test)]
xgb.fit(X_train_mms, y_train ,eval_set = evals, verbose = 50, early_stopping_rounds = 100, eval_metric = 'logloss')


[0]	validation_0-logloss:0.692969
Will train until validation_0-logloss hasn't improved in 100 rounds.
[50]	validation_0-logloss:0.684518
[100]	validation_0-logloss:0.678019
[150]	validation_0-logloss:0.673087
[200]	validation_0-logloss:0.669515
[250]	validation_0-logloss:0.667155
[300]	validation_0-logloss:0.665881
[350]	validation_0-logloss:0.66558
[399]	validation_0-logloss:0.666121


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.001, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=400,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [27]:
voting_func([('DT', dt), ('SVM',svc), ('LR', lr), ('GB', gb), ('XGBoost', xgb)], X_train_mms, X_test_mms)

soft voting
accuracy : 0.6038 precision : 0.7200 recall : 0.5625 F1 : 0.6316 AUC : 0.6146
[[14  7]
 [14 18]]
hard voting
accuracy : 0.6226 precision : 0.6429 recall : 0.8438 F1 : 0.7297 AUC : 0.5647
[[ 6 15]
 [ 5 27]]


In [23]:
voting_func([('DT', dt), ('RF', rf), ('SVM',svc), ('LR', lr), ('GB', gb), ('XGBoost', xgb)], X_train_mms, X_test_mms)

soft voting
accuracy : 0.6792 precision : 0.7586 recall : 0.6875 F1 : 0.7213 AUC : 0.6771
[[14  7]
 [10 22]]
hard voting
accuracy : 0.6226 precision : 0.6304 recall : 0.9062 F1 : 0.7436 AUC : 0.5484
[[ 4 17]
 [ 3 29]]


## MaxAbsScale

In [24]:
#DecisionTree
dt = DecisionTreeClassifier(random_state = 156, max_depth = 8)
dt.fit(X_train_mas, y_train)

#RandomForest
rf = RandomForestClassifier(n_jobs = -1, max_depth=7, min_samples_leaf=8, min_samples_split=16,
                                n_estimators = 200)
rf.fit(X_train_mas, y_train)

#SVM
svc = SVC(kernel='rbf', probability=True)
svc.fit(X_train_mas, y_train)

lr = LogisticRegression()
lr.fit(X_train_mas, y_train)

gb = GradientBoostingClassifier()
gb.fit(X_train_mas, y_train)

xgb = XGBClassifier(learning_rate=0.001, n_estimators=400, n_jobs=-1, max_depth = 7)
evals = [(X_test_mas, y_test)]
xgb.fit(X_train_mas, y_train ,eval_set = evals, verbose = 50, early_stopping_rounds = 100, eval_metric = 'logloss')


[0]	validation_0-logloss:0.693011
Will train until validation_0-logloss hasn't improved in 100 rounds.
[50]	validation_0-logloss:0.686535
[100]	validation_0-logloss:0.681947
[150]	validation_0-logloss:0.678803
[200]	validation_0-logloss:0.676818
[250]	validation_0-logloss:0.675843
[300]	validation_0-logloss:0.675794
[350]	validation_0-logloss:0.676568
Stopping. Best iteration:
[277]	validation_0-logloss:0.675707



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.001, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=400,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [25]:
voting_func([('DT', dt), ('RF', rf), ('SVM',svc), ('LR', lr), ('GB', gb)], X_train_mas, X_test_mas)

soft voting
accuracy : 0.5660 precision : 0.5918 recall : 0.9062 F1 : 0.7160 AUC : 0.4769
[[ 1 20]
 [ 3 29]]
hard voting
accuracy : 0.5660 precision : 0.5918 recall : 0.9062 F1 : 0.7160 AUC : 0.4769
[[ 1 20]
 [ 3 29]]
